In [32]:
import argparse
import os.path as osp
from typing import Any, Dict, Optional

import torch
from torch.nn import (
    BatchNorm1d,
    Embedding,
    Linear,
    ModuleList,
    ReLU,
    Sequential,
)
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINEConv, GPSConv, global_add_pool
from torch_geometric.nn.attention import PerformerAttention

In [33]:
path = '/tmp/ENZYMES'
transform = T.AddRandomWalkPE(walk_length=20, attr_name='pe')
train_dataset = TUDataset(path, name='ENZYMES', pre_transform=transform, force_reload=True)
val_dataset = TUDataset(path, name='ENZYMES', pre_transform=transform, force_reload=True)
test_dataset = TUDataset(path, name='ENZYMES', pre_transform=transform, force_reload=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

Processing...
Done!
Processing...
Done!
Processing...
Done!


In [34]:
class RedrawProjection:
    def __init__(self, model: torch.nn.Module,
                 redraw_interval: Optional[int] = None):
        self.model = model
        self.redraw_interval = redraw_interval
        self.num_last_redraw = 0

    def redraw_projections(self):
        if not self.model.training or self.redraw_interval is None:
            return
        if self.num_last_redraw >= self.redraw_interval:
            fast_attentions = [
                module for module in self.model.modules()
                if isinstance(module, PerformerAttention)
            ]
            for fast_attention in fast_attentions:
                fast_attention.redraw_projection_matrix()
            self.num_last_redraw = 0
            return
        self.num_last_redraw += 1

In [99]:
class GPS(torch.nn.Module):
    def __init__(self, channels: int, pe_dim: int, num_layers: int,
                 attn_type: str):
        super().__init__()

        self.node_emb = Embedding(28, channels - pe_dim)
        self.pe_lin = Linear(20, pe_dim)
        self.pe_norm = BatchNorm1d(20)

        self.convs = ModuleList()
        for _ in range(num_layers):
            nn = Sequential(
                Linear(channels, channels),
                ReLU(),
                Linear(channels, channels),
            )
            conv = GPSConv(channels, GINEConv(nn), heads=4,
                           attn_type=attn_type)
            self.convs.append(conv)

        self.mlp = Sequential(
            Linear(channels, channels // 2),
            ReLU(),
            Linear(channels // 2, channels // 4),
            ReLU(),
            Linear(channels // 4, 1),
        )
        self.redraw_projection = RedrawProjection(
            self.convs,
            redraw_interval=1000 if attn_type == 'performer' else None)

    def forward(self, x, pe, edge_index, batch):
        x_pe = self.pe_norm(pe)
        print(self.node_emb(x.squeeze(-1).long()))
        print(self.node_emb(x.squeeze(-1).long()).mean(dim=1))

        x = torch.cat((self.node_emb(x.squeeze(-1).long()).mean(dim=1), self.pe_lin(x_pe)), 1)
        #edge_attr = self.edge_emb(edge_attr)

        for conv in self.convs:
            x = conv(x, edge_index, batch)
        x = global_add_pool(x, batch)
        return self.mlp(x)


In [100]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
attn_kwargs = {'dropout': 0.5}
model = GPS(channels=64, pe_dim=8, num_layers=10, attn_type="multihead").to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20,
                              min_lr=0.00001)

In [101]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        model.redraw_projection.redraw_projections()
        out = model(data.x, data.pe, data.edge_index, data.batch)
        loss = (out.squeeze() - data.y).abs().mean()
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    total_error = 0
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.pe, data.edge_index, data.batch)
        total_error += (out.squeeze() - data.y).abs().sum().item()
    return total_error / len(loader.dataset)

In [102]:
for epoch in range(1, 101):
    loss = train()
    val_mae = test(val_loader)
    test_mae = test(test_loader)
    scheduler.step(val_mae)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_mae:.4f}, '
          f'Test: {test_mae:.4f}')

tensor([[[-0.5449, -0.3394, -1.1823,  ...,  1.3186, -0.9748,  1.6986],
         [ 0.0493, -0.7118, -0.3606,  ..., -0.1708, -1.1432,  1.4033],
         [ 0.0493, -0.7118, -0.3606,  ..., -0.1708, -1.1432,  1.4033]],

        [[-0.5449, -0.3394, -1.1823,  ...,  1.3186, -0.9748,  1.6986],
         [ 0.0493, -0.7118, -0.3606,  ..., -0.1708, -1.1432,  1.4033],
         [ 0.0493, -0.7118, -0.3606,  ..., -0.1708, -1.1432,  1.4033]],

        [[-0.5449, -0.3394, -1.1823,  ...,  1.3186, -0.9748,  1.6986],
         [ 0.0493, -0.7118, -0.3606,  ..., -0.1708, -1.1432,  1.4033],
         [ 0.0493, -0.7118, -0.3606,  ..., -0.1708, -1.1432,  1.4033]],

        ...,

        [[ 0.0493, -0.7118, -0.3606,  ..., -0.1708, -1.1432,  1.4033],
         [-0.5449, -0.3394, -1.1823,  ...,  1.3186, -0.9748,  1.6986],
         [ 0.0493, -0.7118, -0.3606,  ..., -0.1708, -1.1432,  1.4033]],

        [[ 0.0493, -0.7118, -0.3606,  ..., -0.1708, -1.1432,  1.4033],
         [-0.5449, -0.3394, -1.1823,  ...,  1.3186, -0.

AttributeError: 'NoneType' object has no attribute 'size'